# Лабораторная работа №4
### Задание 1
Провести классификацию найденного датасета, методами наивного Байеса. В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Выполнение работы:
### Шаг 1. Загрузка датасета

In [7]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB,BernoulliNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd


df = pd.read_csv("./base_v3.csv")

y = df["is_fraud"]
X = df.drop("is_fraud", axis=1)

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Шаг 2. Создание модели наивного Байеса


Метод наивного байеса (Naive Bayes) — это семейство простых вероятностных классификаторов, основанных на применении теоремы Байеса со строгими (наивными) предположениями о независимости между признаками.

In [8]:
# Замените StandardScaler на MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создание моделей с соответствующими параметрами
naive_bayes_models = {
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'MultinomialNB': MultinomialNB()
}

# Задание сетки параметров для всех моделей
param_grid = {
    'GaussianNB': {},
    'BernoulliNB': {'alpha': [0.1, 0.5, 1.0], 'binarize': [0.0, 0.1, 0.2]},
    'MultinomialNB': {'alpha': [0.1, 0.5, 1.0]}
}

scoring_metric = 'accuracy'

best_models = {}
for model_name,model in naive_bayes_models.items():
    grid_search = GridSearchCV(model,param_grid[model_name],scoring = scoring_metric,cv=5)
    grid_search.fit(X_train,y_train)

    best_models[model_name] = grid_search.best_estimator_
#Оценка моделей
best_model_name = max(best_models, key=lambda k: grid_search.cv_results_['mean_test_score'][grid_search.best_index_])
best_model = best_models[best_model_name]

print(f"Лучшая модель: {best_model_name}")
print(f"Лучшие параметры: {grid_search.best_params_}")

Лучшая модель: GaussianNB
Лучшие параметры: {'alpha': 0.1}


###  Шаг 3. Вывод результатов


In [9]:
best_model.fit(X_train,y_train)

y_pred_full = best_model.predict(X_test)
accuracy = accuracy_score(y_test,y_pred_full)
classification_rep = classification_report(y_test,y_pred_full)
print(f"Точность модели: {accuracy}")
print(classification_rep)

Точность модели: 0.905
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      2038
           1       0.82      0.47      0.60       362

    accuracy                           0.91      2400
   macro avg       0.87      0.73      0.77      2400
weighted avg       0.90      0.91      0.89      2400

